In [14]:
import cv2
import json
import random
import os
from PIL import Image, ImageEnhance, ImageOps
import numpy as np
#from pycocotools import mask as mask

In [15]:
with open('cat_dog/cat_dog_annotations.json', 'r') as f:
    annotations = json.load(f)
    print(annotations)

{'_via_settings': {'ui': {'annotation_editor_height': 25, 'annotation_editor_fontsize': 0.8, 'leftsidebar_width': 18, 'image_grid': {'img_height': 80, 'rshape_fill': 'none', 'rshape_fill_opacity': 0.3, 'rshape_stroke': 'yellow', 'rshape_stroke_width': 2, 'show_region_shape': True, 'show_image_policy': 'all'}, 'image': {'region_label': 'class', 'region_color': '__via_default_region_color__', 'region_label_font': '10px Sans', 'on_image_annotation_editor_placement': 'NEAR_REGION'}}, 'core': {'buffer_size': 18, 'filepath': {}, 'default_filepath': ''}, 'project': {'name': 'via_project_18Oct2022_11h49m'}}, '_via_img_metadata': {'1.jpeg45602': {'filename': '1.jpeg', 'size': 45602, 'regions': [{'shape_attributes': {'name': 'rect', 'x': 284, 'y': 27, 'width': 365, 'height': 478}, 'region_attributes': {'class': {'dog': True}}}], 'file_attributes': {}}, '2.jpeg4669': {'filename': '2.jpeg', 'size': 4669, 'regions': [{'shape_attributes': {'name': 'rect', 'x': 6, 'y': 39, 'width': 150, 'height': 107

In [16]:
annotations.keys()

dict_keys(['_via_settings', '_via_img_metadata', '_via_attributes'])

In [17]:
def rotate_bbox(bbox, img_width, img_height, angle):
    radians = angle * np.pi / 180.0
    cos = np.cos(radians)
    sin = np.sin(radians)

    x1, y1, x2, y2 = bbox

    #translate box center origin
    x1 -= img_width / 2
    y1 -= img_height / 2
    x2 -= img_width / 2
    y2 -= img_height /2

    #rotate box coordinates
    x1r = x1 * cos - y1 * sin
    y1r = x1 * sin + y1 * cos
    x2r = x2 * cos - y2 * sin
    y2r = x2 * sin + y2 * cos

    #translate box center back to original position
    x1r += img_width / 2
    y1r += img_height / 2
    x2r += img_width / 2
    y2r += img_height / 2

    return [int(x1r), int(y1r), int(x2r), int(y2r)]


In [18]:
input_dir = 'cat_dog'
output_dir = 'output images'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
count = 0

# load annotations
with open('cat_dog/cat_dog_annotations.json', 'r') as f:
    annotations = json.load(f)

# loop through images
for img_id, img_info in annotations['_via_img_metadata'].items():
    img_path = os.path.join(input_dir, img_info['filename'])
    img = cv2.imread(img_path)

    if img is None:
        print(f'Error reading image {img_path}')
        continue

    bbox_list = []
    for region in img_info['regions']:
        x = region['shape_attributes']['x']
        y = region['shape_attributes']['y']
        w = region['shape_attributes']['width']
        h = region['shape_attributes']['height']
        bbox_list.append([x, y, x+w, y+h])

    # apply resize and brightness enhancement
    resize_scale = random.uniform(0.6, 1.0)
    factor = random.uniform(0.5, 1.5)

    h, w = img.shape[:2]
    new_h = int(h * resize_scale)
    new_w = int(w * resize_scale)
    img_resized = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_LINEAR)

    img_pil = Image.fromarray(np.uint8(img_resized))
    enhancer = ImageEnhance.Brightness(img_pil)
    img_pil = enhancer.enhance(factor)
    img_resized = cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

    # loop through angles and rotate images
    for angle in range(0, 360, 5):
        rows, cols = img_resized.shape[:2]
        M = cv2.getRotationMatrix2D((cols/2, rows/2), angle, 1)
        img_rotated = cv2.warpAffine(img_resized, M, (cols, rows))
        rotated_bbox = [rotate_bbox(bbox, img_resized.shape[1], img_resized.shape[0], angle) for bbox in bbox_list]


        # save rotated image and annotation
        filename = f'{count}.jpg'
        output_path = os.path.join(output_dir, filename)
        cv2.imwrite(output_path, img_rotated)

        rotated_annotations = {
            '_via_img_metadata': {
                filename: {
                    'filename': filename,
                    'regions': [],
                    'size': img_rotated.shape[:2]
                }
            }
        }

        for bbox in rotated_bbox:
            x1, y1, x2, y2 = bbox
            rotated_annotations['_via_img_metadata'][filename]['regions'].append({
                'shape_attributes': {
                    'name': 'rect',
                    'x': x1,
                    'y': y1,
                    'width': x2 - x1,
                    'height': y2 - y1
                },
                'region_attributes': {}
            })                        
                        




In [ ]:
# input_dir = 'cat_dog'
# output_dir = 'output images'
# if not os.path.exists(output_dir):
#     os.makedirs(output_dir)
# count = 0

# for img_id, img_info in annotations['_via_img_metadata'].items():
#     img_path = os.path.join(input_dir, img_info['filename'])
#     img = cv2.imread(img_path)

#     if img is None:
#         print(f'Error reading image {img_path}')
#         continue

#     bbox_list = []
#     for region in img_info['regions']:
#         x = region['shape_attributes']['x']
#         y = region['shape_attributes']['y']
#         w = region['shape_attributes']['width']
#         h = region['shape_attributes']['height']
#         bbox_list.append([x, y, x+w, y+h])

#     for angle in range(0, 360, 360//7):
#         #rotate the image
#         rows, cols, = img.shape[:2]
#         M = cv2.getRotationMatrix2D((cols/2, rows/2), angle, 1)
#         img_rotated = cv2.warpAffine(img, M, (cols, rows))
#         rotated = [rotate_bbox(bbox, img.shape[1], img.shape[0], angle) for bbox in bbox_list]
#         bbox_rotated = [rotate_bbox(bbox, img.shape[1], img.shape[0], angle) for bbox in bbox_list]



#         resize_scale = random.uniform(0.6, 1.0)
#         h, w = img.shape[:2]
#         new_h = int(h * resize_scale)
#         new_w = int(w * resize_scale)
#         img_resized = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_LINEAR)

#         bbox_resized = []
#         for bbox in bbox_list:
#             xmin, ymin, xmax, ymax = bbox
#             xmin_resize = int(xmin * resize_scale)
#             ymin_resize = int(ymin * resize_scale)
#             xmax_resize = int(xmax * resize_scale)
#             ymax_resize = int(ymax * resize_scale)
#             bbox_resized.append([xmin_resize, ymin_resize, xmax_resize, ymax_resize])


#         for bbox in bbox_resized:
#             xmin, ymin, xmax, ymax = bbox
#             cv2.rectangle(img_resized, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
            
#         img_pil = Image.fromarray(np.uint8(img_resized))
#         factor = random.uniform(0.5, 1.5)
#         enhancer = ImageEnhance.Brightness(img_pil)
#         img_pil = enhancer.enhance(factor)
#         img_resized = cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)
        
#         output_path = os.path.join(output_dir, f"{img_id}_{angle}.jpg")
#         cv2.imwrite(output_path, img_resized)
#         if os.path.exists(output_path):
#             print(f"Image saved to {output_path}")
#         else:
#             print(f"Error saving image to {output_path}")

#         count += 1
#         if count >= 504:
#             break